# Ecodatalab2 - Using Weighted Average Method to Generate Estimates for Electricity Dataset

Author: Jiaxin Kathy Li and Christopher Jones 

Summer 2020

In [8]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')




# Related Files and Preparation: 
# Relationship file, filled Elec_Master dataset and some cleaning and preliminary operations on them

Firstly, the dataset we will take advantage of is the census relationship files that include zip code population, population distribution ratio(s) within a zip code to counties, GEOID, etc. This would be the dataset we will use as the translator and connector between datasets. 

Secondly, the dataset that we are trying to fill is the elec_master speadsheet that former EcoDataLab staff had composed. It includes zip code, year, customer class(residential, industrial, commercial) and electricity consumption for each month as well the year average. Notably, if there is at least one nan value in the monthly data, the AnyNull? column would be True instead of False. Also, we would be working with residential dataset in this part since number of costumers for residential data would be population, in order to simplify our problem. Using simple average method, we have already generated estimates for all missing values. We will continue our calculations based on that. 

Lastly, we would extract partial dataset from relationship file that is needed for the operations on Elec_Master by creating intermediate dataframes:
    1. ca_relationship: relationship file with on Californian zip codes extant in the Elec_Master dataset
    2. county_to_zcta: link each county code with a list of zip codes that are in this county
    3. zcta_to_county: link each zipcode with the county(counties) it is in/spanning. 
    4. zcta_pop: extract from relationship file, the zip code and corresponding population
    
There are other sanity checks and list_of_spanning_zctas that includes the zip codes that span multiple counties for test use. 

In [9]:
relationship = pd.read_csv('zcta_county_rel_10.txt')
relationship.head(2)

,ZCTA5,STATE,COUNTY,GEOID,POPPT,HUPT,AREAPT,AREALANDPT,ZPOP,ZHU,...,COAREA,COAREALAND,ZPOPPCT,ZHUPCT,ZAREAPCT,ZAREALANDPCT,COPOPPCT,COHUPCT,COAREAPCT,COAREALANDPCT
0,601,72,1,72001,18465,7695,165132671,164333375,18570,7744,...,173777444,172725651,99.43,99.37,98.61,98.6,94.77,94.71,95.03,95.14
1,601,72,141,72141,105,49,2326414,2326414,18570,7744,...,298027589,294039825,0.57,0.63,1.39,1.4,0.32,0.35,0.78,0.79


In [10]:
elec_mr_filled = pd.read_csv('weighted _average_estimate_with_weighted_population_fraction.csv')
elec_mr_filled.head(2)

,ZCTA,Year,CustomerClass,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,AnnualTotalAverage,anyNull?
0,90001,2014,residential,348.518658,307.625913,309.033842,288.414703,292.111064,336.951066,436.061225,390.115486,337.090451,372.627938,308.556853,355.073121,1036.257911,True
1,90001,2015,residential,394.008751,309.371632,303.939084,310.553367,297.245193,311.423576,376.909580,380.741922,436.034409,415.524191,329.881948,405.894790,4271.528443,False


In [75]:
# narrowing the relationship files to the zipcodes that are extant in elem_m
zcta_in_elec_m = elec_mr_filled['ZCTA'].unique()
ca_relationship = relationship[relationship['ZCTA5'].isin(zcta_in_elec_m)]

# check if total number of zcta matches: TRUE
len(zcta_in_elec_m) == len(ca_relationship['ZCTA5'].unique())

# check the state is correct in ca_relationship
len(ca_relationship.groupby('STATE').agg('size').index) == 1

# extract list of counties and corresponding zip codes
len(ca_relationship['COUNTY'].unique()) # note: there are 56 counties recorded in ca_relationship not 58 in reality 
county_to_zcta = ca_relationship[['ZCTA5','COUNTY']].groupby('COUNTY').agg(list)
zcta_to_county = ca_relationship[['ZCTA5','COUNTY']].groupby('ZCTA5').agg(list)
# note: each zcta matches with 1-4 counties
# ca_relationship[['ZCTA5','COUNTY']].groupby('ZCTA5').agg(len)['COUNTY'].value_counts()

# extract population with regards to zipcode
zcta_pop_dup = ca_relationship[['ZCTA5', 'ZPOP']]
zcta_pop = zcta_pop_dup.groupby('ZCTA5').agg(np.mean).reset_index()

# county population: consider zip code spanning
ca_county_pop = ca_relationship[['ZCTA5', 'COUNTY', 'POPPT']].groupby('COUNTY').agg(list)
ca_county_pop['COUNTY POP'] = ca_relationship[['COUNTY', 'POPPT']].groupby('COUNTY').agg(sum)
county_pop = ca_county_pop[['ZCTA5','COUNTY POP']]
# sanity check: our county table does take spanning zip code into consideration 
sample_zips = ca_county_pop.loc[1][0]
sum(ca_relationship[ca_relationship['ZCTA5'].isin(zips)]['ZPOP']) == ca_county_pop.loc[1][1]

False

In [17]:
county_to_zcta.head(2)

,ZCTA5
COUNTY,
1,"[94505, 94514, 94536, 94538, 94539, 94541, 945..."
3,[95223]


In [18]:
zcta_to_county.head(2)

,COUNTY
ZCTA5,
90001,[37]
90002,[37]


In [19]:
zcta_pop.head(2)

,ZCTA5,ZPOP
0,90001,57110
1,90002,51223


In [77]:
county_pop.head(2)

,ZCTA5,COUNTY POP
COUNTY,,
1,"[94505, 94514, 94536, 94538, 94539, 94541, 945...",1435704
3,[95223],121


# Methodology Explanation 

Former calculations using the Simple Average Method show average monthly electricity consumption per customer (household) for each zip code. However, some zip codes cross multiple counties. Census relationship files show the total number of households (HUPT) from each zip code living in each county in the year 2010. We want to create a weighted average household consumption for each county. Here are our steps:

1. Multiply average monthly electricity by number of households in each zip code that live within the county (from the relationship file)
2. Sum up total kWh for each county
3. Divide by the total number of households in each county (from the relationship file)



First, we would construct a table that contains sum of electricity consumption per month, per zipcode.

In [107]:
monthly_avg_prep = elec_weighted_prep.merge(zcta_pop,left_on='ZCTA',right_on='ZCTA5',how='left').drop(
    columns={'ZCTA5'})
monthly_total_byzcta = monthly_avg_prep.iloc[:, :2]

months = [str(x) for x in np.arange(1.,13.,1)]

for mon in months:
    to_multiple = monthly_avg_prep[mon]
    series = to_multiple*monthly_avg_prep['ZPOP']
    monthly_total_byzcta[mon] = series
    
monthly_total_byzcta.head()

,ZCTA,Year,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
0,90001,2014,1.990390e+07,1.756852e+07,1.764892e+07,1.647136e+07,1.668246e+07,1.924328e+07,2.490346e+07,2.227950e+07,1.925124e+07,2.128078e+07,1.762168e+07,2.027823e+07
1,90001,2015,2.250184e+07,1.766821e+07,1.735796e+07,1.773570e+07,1.697567e+07,1.778540e+07,2.152531e+07,2.174417e+07,2.490193e+07,2.373059e+07,1.883956e+07,2.318065e+07
2,90001,2016,2.388566e+07,1.903103e+07,1.779362e+07,1.671362e+07,1.621921e+07,1.928954e+07,2.235260e+07,2.304389e+07,2.155534e+07,2.017012e+07,1.951097e+07,2.115992e+07
3,90001,2017,2.318241e+07,2.119579e+07,1.744982e+07,1.661554e+07,1.679152e+07,1.909069e+07,2.367567e+07,2.343655e+07,2.464523e+07,1.959945e+07,1.977787e+07,1.951019e+07
4,90001,2018,2.138485e+07,2.020760e+07,2.234399e+07,1.700302e+07,1.609134e+07,1.859043e+07,2.564086e+07,2.919572e+07,2.444849e+07,1.897519e+07,1.882926e+07,2.011946e+07


Then, we would group by county and find weighted county sum.

In [108]:
monthly_total_byzcta_withcounty = monthly_total_byzcta.merge(zcta_to_county.reset_index(),
                                                 left_on='ZCTA',right_on='ZCTA5',how='left').drop(columns={'ZCTA5'})
monthly_total_byzcta_withcounty.head()

,ZCTA,Year,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,COUNTY
0,90001,2014,1.990390e+07,1.756852e+07,1.764892e+07,1.647136e+07,1.668246e+07,1.924328e+07,2.490346e+07,2.227950e+07,1.925124e+07,2.128078e+07,1.762168e+07,2.027823e+07,[37]
1,90001,2015,2.250184e+07,1.766821e+07,1.735796e+07,1.773570e+07,1.697567e+07,1.778540e+07,2.152531e+07,2.174417e+07,2.490193e+07,2.373059e+07,1.883956e+07,2.318065e+07,[37]
2,90001,2016,2.388566e+07,1.903103e+07,1.779362e+07,1.671362e+07,1.621921e+07,1.928954e+07,2.235260e+07,2.304389e+07,2.155534e+07,2.017012e+07,1.951097e+07,2.115992e+07,[37]
3,90001,2017,2.318241e+07,2.119579e+07,1.744982e+07,1.661554e+07,1.679152e+07,1.909069e+07,2.367567e+07,2.343655e+07,2.464523e+07,1.959945e+07,1.977787e+07,1.951019e+07,[37]
4,90001,2018,2.138485e+07,2.020760e+07,2.234399e+07,1.700302e+07,1.609134e+07,1.859043e+07,2.564086e+07,2.919572e+07,2.444849e+07,1.897519e+07,1.882926e+07,2.011946e+07,[37]


In [145]:
ca_county_pop_relationship = ca_relationship[['ZCTA5','COUNTY','ZPOPPCT']]
ca_county_pop_relationship['ZPOPPCT'] = ca_county_pop_relationship['ZPOPPCT']*0.01
ca_county_pop_relationship.head()

/srv/conda/envs/data100/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,ZCTA5,COUNTY,ZPOPPCT
40937,90001,37,1.0
40938,90002,37,1.0
40944,90008,37,1.0
40957,90022,37,1.0
40958,90023,37,1.0


In [251]:
# test round for 2016, county 41
year=2014
county =111

df = pd.DataFrame(index=(['Year']+months))
all_data = monthly_total_byzcta_withcounty[(monthly_total_byzcta_withcounty['Year']==year)&
                                                   ([county in x for x in monthly_total_byzcta_withcounty['COUNTY']])].drop(columns={'COUNTY'}) 
county_pop_relationship_here = ca_county_pop_relationship[ca_county_pop_relationship['COUNTY']==county]
all_merged = all_data.merge(county_pop_relationship_here,
                           left_on='ZCTA', right_on='ZCTA5', how='left').drop(columns={'ZCTA5'})
for mon in months:
    all_merged[mon] = all_merged[mon] * all_merged['ZPOPPCT']
    mon = all_merged[months]  
    column_this_county = np.append(np.array(year),np.array(np.sum(mon, axis=0).reset_index()[0]))
    df[county] = column_this_county
df    

,111
Year,2.014000e+03
1.0,3.674159e+08
2.0,3.147036e+08
3.0,3.167543e+08
4.0,3.320060e+08
5.0,3.985674e+08
6.0,4.953207e+08
7.0,6.137510e+08
8.0,5.803777e+08
9.0,4.907096e+08


In [258]:
years = monthly_total_byzcta['Year'].unique()
# monthly_total_bycounty = monthly_total_byzcta_withcounty.copy().loc[:0, :].drop(0).rename(columns={'ZCTA':'COUNTY'})
counties = ca_relationship['COUNTY'].unique()
monthly_total_bycounty_T=pd.DataFrame(index=(['Year']+months))


for year in years:
    for county in counties: 
        all_data = monthly_total_byzcta_withcounty[(monthly_total_byzcta_withcounty['Year']==year)&
                                                   ([county in x for x in monthly_total_byzcta_withcounty['COUNTY']])].drop(columns={'COUNTY'})  
        county_pop_relationship_here = ca_county_pop_relationship[ca_county_pop_relationship['COUNTY']==county]
        all_merged = all_data.merge(county_pop_relationship_here,
                           left_on='ZCTA', right_on='ZCTA5', how='left').drop(columns={'ZCTA5'})
        for mon in months:
            all_merged[mon] = all_merged[mon] * all_merged['ZPOPPCT']
            months_only = all_merged[months]  
            column_this_county = np.append(np.array(year),np.array(np.sum(months_only, axis=0).reset_index()[0]))
            monthly_total_bycounty_T[str(year)+str(county)] = column_this_county
            

In [259]:
monthly_total_bycounty_T.head()

,201437,2014111,201459,201471,201465,201473,201425,201427,201483,201431,...,201311,2013115,20137,201391,201321,201363,2013103,201389,201335,201349
Year,2.014000e+03,2.014000e+03,2.014000e+03,2.014000e+03,2.014000e+03,2.014000e+03,2014.000000,2014.000000,2.014000e+03,2.014000e+03,...,2.013000e+03,2.013000e+03,2.013000e+03,2013.000000,2.013000e+03,2.013000e+03,2.013000e+03,2.013000e+03,2.013000e+03,2013.000000
1.0,3.468143e+09,3.674159e+08,1.615803e+09,1.234616e+08,1.306135e+09,1.572042e+09,65513.402951,10091.000288,1.927758e+08,1.138890e+08,...,1.535025e+07,4.871050e+07,1.541445e+08,460546.630775,2.312311e+07,1.008405e+07,5.241568e+07,1.703229e+08,1.841270e+06,112412.621103
2.0,3.054700e+09,3.147036e+08,1.400856e+09,1.087486e+08,1.150371e+09,1.363044e+09,81974.659850,8217.627010,1.709758e+08,9.284411e+07,...,1.210663e+07,3.935559e+07,1.244482e+08,395193.739447,1.812512e+07,7.963694e+06,4.084372e+07,1.352244e+08,1.409760e+06,80916.357021
3.0,3.071343e+09,3.167543e+08,1.471854e+09,1.088406e+08,1.152309e+09,1.391718e+09,88371.288929,8363.559750,1.750180e+08,9.057573e+07,...,1.175346e+07,3.889653e+07,1.204729e+08,417043.422908,1.739280e+07,7.361604e+06,3.947702e+07,1.310815e+08,1.328768e+06,75188.553905
4.0,2.889238e+09,3.320060e+08,1.423492e+09,1.023220e+08,1.083755e+09,1.303567e+09,57802.607348,9102.197481,1.657579e+08,9.368843e+07,...,1.173919e+07,3.870924e+07,1.157761e+08,397920.822135,1.678720e+07,6.213993e+06,3.744659e+07,1.206172e+08,1.222718e+06,66434.118030


We would next transpose our table and do some cleaning.

In [270]:
monthly_total_bycounty = monthly_total_bycounty_T.transpose().reset_index().rename(columns={'index':'County'})
monthly_total_bycounty['County'] =[int(x) for x in monthly_total_bycounty['County'].str[4:]]
monthly_total_bycounty['Year'] = [int(x) for x in monthly_total_bycounty['Year']]
monthly_total_bycounty.head()

,County,Year,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0
0,37,2014,3.468143e+09,3.054700e+09,3.071343e+09,2.889238e+09,2.972038e+09,3.431739e+09,4.430673e+09,3.992997e+09,3.469422e+09,3.899798e+09,3.105393e+09,3.262440e+09
1,111,2014,3.674159e+08,3.147036e+08,3.167543e+08,3.320060e+08,3.985674e+08,4.953207e+08,6.137510e+08,5.803777e+08,4.907096e+08,4.705096e+08,4.007328e+08,4.407847e+08
2,59,2014,1.615803e+09,1.400856e+09,1.471854e+09,1.423492e+09,1.704957e+09,1.483283e+09,2.036997e+09,2.132923e+09,2.538137e+09,1.715442e+09,1.355029e+09,1.453330e+09
3,71,2014,1.234616e+08,1.087486e+08,1.088406e+08,1.023220e+08,1.080059e+08,1.064742e+08,1.391497e+08,1.431508e+08,1.594237e+08,1.411575e+09,1.005097e+09,1.049750e+09
4,65,2014,1.306135e+09,1.150371e+09,1.152309e+09,1.083755e+09,1.148960e+09,1.122232e+09,1.468855e+09,1.519832e+09,1.709390e+09,1.664455e+09,1.132340e+09,1.144169e+09


In [274]:
monthly_avg_bycounty = monthly_total_bycounty.merge(county_pop.reset_index()[['COUNTY','COUNTY POP']],
                                                   left_on='County', right_on='COUNTY', how='left').drop(columns={'COUNTY'})
for mon in months:
    monthly_avg_bycounty[mon] = monthly_avg_bycounty[mon] / monthly_avg_bycounty['COUNTY POP']
    
monthly_avg_bycounty.head()


,County,Year,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0,12.0,COUNTY POP
0,37,2014,522.007731,459.778292,462.283381,434.873854,447.336366,516.528350,666.883002,601.006098,522.200286,586.978241,467.408362,491.046341,6643854
1,111,2014,446.504291,382.445376,384.937459,403.472268,484.361388,601.941383,745.864461,705.307330,596.337753,571.789576,486.992866,535.666222,822872
2,59,2014,581.681334,504.301573,529.860636,512.450451,613.776380,533.974929,733.309390,767.842209,913.717438,617.551031,487.803990,523.192054,2777814
3,71,2014,61.586057,54.246814,54.292665,51.041046,53.876300,53.112242,69.411693,71.407533,79.524928,704.132543,501.370241,523.644320,2004701
4,65,2014,675.517455,594.958175,595.960320,560.505110,594.228424,580.405137,759.673946,786.038657,884.076053,860.835923,585.632752,591.750550,1933533


To summarize what we have done in the above steps, we have turned our filled dataset of elec_master_residential that bases the data on zipcodes into the dataset that bases on county, taking spanning zip codes into consideration. Therefore, each entry in the table is the average electricity consumption for the county.

In [275]:
monthly_avg_bycounty.to_csv('monthly_avg_bycounty.csv')